### Train a Machine Lerning Models with Azure ML

Training a machine learning algorithm can be very easy but the production environment can bring more complexity to the problem, such as:
- Create reusable training scripts for multiple compute environment.
- Logging model performance metrics during training and evaluation process.
- Tracking model version.

Training a machine learning models can be done through ScriptRunConfig and create a new experiment.
A Conda environment must be set along with all the needed packages.

In [11]:
from azureml.core import Workspace

ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

#### Train Inline

In Azure Machine Learning, after the training of a model, we can retrieve some information by:
- Overview
- Metrics in which the log values are stored
- Images in which the logged images are stored.
- Child jobs
- Output + logs
- Code: the folder above the current notebook.
- Explanations
- Fairness helps us to visualize predictive performance disparities, namely if there are some biases.

In [45]:
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

run = Run.get_context()

data = pd.read_csv('data/diabetes.csv')
X, y = data.drop(['Diabetic', 'PatientID'], axis = 1).values, data.Diabetic.values
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 0, test_size = .3)

reg = 0.1
model = LogisticRegression(C = 1/reg, solver = 'liblinear').fit(X_train, y_train)

y_hat = model.predict(X_val)
acc = np.average(y_hat == y_val)

run.log('Accuracy:', acc)
joblib.dump(value=model, filename='outputs/model.pkl')

os.makedirs('outputs', exist_ok = True)

run.complete()

Attempted to log scalar metric Accuracy::
0.774


#### Train with Script

Data must be stored within the same directory of the script.

In [52]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration

# Create a Python environment for the experiment
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory='Script',
                                script='2_Train.py',
                                environment=env,
                                compute_target = 'my-compute',
                                docker_runtime_config=DockerConfiguration(use_docker=True))

# Submit the experiment
experiment = Experiment(workspace=ws, name='training-experiment')
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output = True)

RunId: training-experiment_1671397869_0c87d34e
Web View: https://ml.azure.com/runs/training-experiment_1671397869_0c87d34e?wsid=/subscriptions/d12c1b85-0a70-4232-b483-12d1ffcfc148/resourcegroups/ResourceGroupRavazzi/workspaces/aml-workspace&tid=b00367e2-193a-4f48-94de-7245d45c0947

Streaming user_logs/std_log.txt

/azureml-envs/azureml_809a074975457de1dd27bdfcf2d79d61/lib/python3.6/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend
-c:159: FutureWarning: azureml.core: AzureML support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use AzureML will continue to work without modification, but Python 3.6 users will no longer get access to the latest AzureML features and bugfixes. We r

{'runId': 'training-experiment_1671397869_0c87d34e',
 'target': 'my-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-12-18T21:11:21.604846Z',
 'endTimeUtc': '2022-12-18T21:11:34.92426Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '165fd291-6703-4d01-97f8-d6058dd8f8fe',
  'azureml.git.repository_uri': 'https://github.com/LuciaRavazzi/AzureML.git',
  'mlflow.source.git.repoURL': 'https://github.com/LuciaRavazzi/AzureML.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'df4c86328c098a60581020fc4d0fc488fee33532',
  'mlflow.source.git.commit': 'df4c86328c098a60581020fc4d0fc488fee33532',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': '2_Train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],

In [51]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.01
Accuracy 0.774
AUC 0.8483441962286681


outputs/diabetes_model.pkl
system_logs/cs_capability/cs-capability.log
system_logs/hosttools_capability/hosttools-capability.log
system_logs/lifecycler/execution-wrapper.log
system_logs/lifecycler/lifecycler.log
system_logs/lifecycler/vm-bootstrapper.log
system_logs/metrics_capability/metrics-capability.log
system_logs/snapshot_capability/snapshot-capability.log
user_logs/std_log.txt


In [55]:
# Models can be saved whitin the output folder but also be registered in the
# AzureML workspace.

from azureml.core import Model

# tags can be modified after the registering step, for proedure doesn't work in that way.
run.register_model(model_path = 'outputs/diabetes_model.pkl',
                   model_name = 'diabetes_model',
                   tags = {'Training context': 'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

Model(workspace=Workspace.create(name='aml-workspace', subscription_id='d12c1b85-0a70-4232-b483-12d1ffcfc148', resource_group='ResourceGroupRavazzi'), name=diabetes_model, id=diabetes_model:1, version=1, tags={'Training context': 'Script'}, properties={'AUC': '0.8483441962286681', 'Accuracy': '0.774'})

In [56]:
# List registered models.

for model in Model.list(ws):
    print(model.name, model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model 1
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774




#### Train with Script and parser

Some hyperparameters can be passed to a function as external parameter through the CLI.

In [62]:
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory='Script',
                                script='4_Train_with_parameters.py',
                                arguments=['--reg-rate', 0.5],
                                environment=env,
                                compute_target = 'my-compute',
                                docker_runtime_config=DockerConfiguration(use_docker=True))

# Submit the experiment
experiment = Experiment(workspace=ws, name='training-experiment')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion(show_output = True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: training-experiment_1671398623_e5e69275
Web View: https://ml.azure.com/runs/training-experiment_1671398623_e5e69275?wsid=/subscriptions/d12c1b85-0a70-4232-b483-12d1ffcfc148/resourcegroups/ResourceGroupRavazzi/workspaces/aml-workspace&tid=b00367e2-193a-4f48-94de-7245d45c0947

Streaming user_logs/std_log.txt

/azureml-envs/azureml_809a074975457de1dd27bdfcf2d79d61/lib/python3.6/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend
-c:159: FutureWarning: azureml.core: AzureML support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use AzureML will continue to work without modification, but Python 3.6 users will no longer get access to the latest AzureML features and bugfixes. We r

{'runId': 'training-experiment_1671398623_e5e69275',
 'target': 'my-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-12-18T21:23:51.753929Z',
 'endTimeUtc': '2022-12-18T21:24:04.982913Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '0ed5c375-22d0-4d2a-817a-9f9b5813fef8',
  'azureml.git.repository_uri': 'https://github.com/LuciaRavazzi/AzureML.git',
  'mlflow.source.git.repoURL': 'https://github.com/LuciaRavazzi/AzureML.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'df4c86328c098a60581020fc4d0fc488fee33532',
  'mlflow.source.git.commit': 'df4c86328c098a60581020fc4d0fc488fee33532',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': '4_Train_with_parameters.py',
  'command': '',
  'useAbsolutePath': False,
 

In [65]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.5
Accuracy 0.774
AUC 0.84822428976566


outputs/diabetes_model.pkl
outputs/my_diabetes_model.pkl
system_logs/cs_capability/cs-capability.log
system_logs/hosttools_capability/hosttools-capability.log
system_logs/lifecycler/execution-wrapper.log
system_logs/lifecycler/lifecycler.log
system_logs/lifecycler/vm-bootstrapper.log
system_logs/metrics_capability/metrics-capability.log
system_logs/snapshot_capability/snapshot-capability.log
user_logs/std_log.txt


In [79]:
from azureml.core import Model

# A model can be registered by means of a local file or by the run method.

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='Script/outputs/diabetes_model.pkl', # local path
                       description='A classification model',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Registering model classification_model


In [66]:
# Register the model.
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})


Model(workspace=Workspace.create(name='aml-workspace', subscription_id='d12c1b85-0a70-4232-b483-12d1ffcfc148', resource_group='ResourceGroupRavazzi'), name=diabetes_model, id=diabetes_model:2, version=2, tags={'Training context': 'Parameterized script'}, properties={'AUC': '0.84822428976566', 'Accuracy': '0.774'})

In [67]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 2
	 Training context : Parameterized script
	 AUC : 0.84822428976566
	 Accuracy : 0.774


diabetes_model version: 1
	 Training context : Script
	 AUC : 0.8483441962286681
	 Accuracy : 0.774


